In [249]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, AveragePooling2D, Flatten
from scikeras.wrappers import KerasClassifier
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.utils import to_categorical

## For plotting
import matplotlib.pyplot as plt
from seaborn import set_style

## This sets the plot style
## to have a grid on a white background
set_style("darkgrid")

In [212]:
##load data
X = np.load('../../data/processed_data/specgram_db_pad_trunc.npy')
## We need to reshape our data to be the correct dimensions for
## the net
## observations, grid, colorvalues

##so we just need to add the colorvalues, 1 here represents single valued
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
print(X.shape)

df = pd.read_csv("../../data/processed_data/metadata.csv")
category_to_number = {'brushing': 0, 'food': 1, 'isolation': 2}

# Create a new column with numerical values based on the mapping
df['numerical_situation'] = df['situation'].map(category_to_number)

y = df['numerical_situation'].values

(440, 128, 67, 1)


In [215]:
## parameter for our data
num_classes = 3
(height, width, channels) = (128,67,1)

In [216]:
#train,test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                       test_size=.2,
                                                       shuffle=True,
                                                       random_state=100,
                                                       stratify=y)



In [217]:
##CNN model in a function
def create_CNN_model(optimizer='adam', filters=32, kernel_size=(3,3), pool_size=(2, 2), num_layers=1, num_dense_layers=1, dense_units=64):
    
    model = models.Sequential()
    
    for _ in range(num_layers):
        model.add(Conv2D(filters*(_+1), kernel_size=kernel_size, activation='relu', input_shape=(height, width, channels)))
        model.add(MaxPooling2D(pool_size=pool_size, strides=2))
        
    model.add(Flatten())
    
    for _ in range(num_dense_layers):
        model.add(Dense(units=dense_units, activation='relu'))
    
    model.add(Dense(units=num_classes, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model




In [218]:
## plot history to check overfitting
def plottv(history_dict,epochs):
    plt.figure(figsize = (8,6))

    plt.scatter(range(1,epochs+1), history_dict['accuracy'], label = "Training Accuracy")
    plt.scatter(range(1,epochs+1), history_dict['val_accuracy'], label = "Validation Set Accuracy")

    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Accuracy", fontsize=12)

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    plt.legend(fontsize=12)

    plt.show()


    plt.figure(figsize = (8,6))

    plt.scatter(range(1,epochs+1), history_dict['loss'], label = "Training Loss")
    plt.scatter(range(1,epochs+1), history_dict['val_loss'], label = "Validation Set Loss")

    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Loss Function Value", fontsize=12)

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    plt.legend(fontsize=12)

    plt.show()

In [219]:

# Set up the parameter grid for GridSearchCV
param_grid = {
    'batch_size':[100],
    'optimizer': ['adam'],#'adagrad','adadelta','sgd','adam',],
    'filters': [32],
    'kernel_size': [(6,3)],#,(2,2),(4,2),(3,3),(6, 3)],
    'pool_size': [(2,2)],#,(3, 2)],#,(3, 3),(4,3)],
    'num_layers': [3],  # Specify the range or list of possible values for the number of conv/pooling layers
    'num_dense_layers': [1]
}




In [220]:
best_accuracy = 0.0
best_params = {}
epochs = 60
# Perform GridSearchCV manually
for batch_size in param_grid['batch_size']:
    for optimizer in param_grid['optimizer']:
        for filters in param_grid['filters']:
            for kernel_size in param_grid['kernel_size']:
                for pool_size in param_grid['pool_size']:
                    for num_layers in param_grid['num_layers']:
                        for num_dense_layers in param_grid['num_dense_layers']:
                            
                            model = create_CNN_model(optimizer=optimizer, filters=filters, kernel_size=kernel_size,
                                                 pool_size=pool_size, num_layers=num_layers, num_dense_layers=num_dense_layers)


                            keras_classifier = KerasClassifier(model=lambda: model, epochs=epochs, batch_size=batch_size, verbose=0)
                            #print(keras_classifier.model)
                            # Perform cross-validation
                            scores = []
                            round = 0
                            for train_index, val_index in cv.split(X_train, y_train):
                                round += 1
                                print("round ",round)
                                X_tt, X_ho = X_train[train_index], X_train[val_index]
                                y_tt, y_ho = y_train[train_index], y_train[val_index]
                                history = keras_classifier.fit(X_tt, to_categorical(y_tt, num_classes=num_classes),validation_data=(X_ho, to_categorical(y_ho, num_classes=num_classes)))
                                history_dict = history.history_
                                plottv(history_dict,epochs)
                                score = keras_classifier.score(X_ho, to_categorical(y_ho))
                                scores.append(score)
                                scores.append(history_dict['val_accuracy'])
                                
                            print(scores)
                            mean_score = np.mean(scores)

                            # Check if the current parameter combination gives a higher accuracy
                            if mean_score > best_accuracy:
                                best_accuracy = mean_score
                                best_params = {
                                    'optimizer': optimizer,
                                    'filters': filters,
                                    'kernel_size': kernel_size,
                                    'pool_size': pool_size,
                                    'num_layers': num_layers,
                                    'num_dense_layers': num_dense_layers
                                }

# Print the results
print("Best parameters found: ", best_params)
print("Best accuracy found: {:.2f}%".format(best_accuracy*100))


round  1


KeyboardInterrupt: 

In [ ]:
# Set up the parameter grid for GridSearchCV
outparam_grid = {
    'batch_size':[100],
    'epochs':[30],
    'optimizer': ['adam'],#'adagrad','adadelta','sgd','adam',],
    'filters': [32],
    'kernel_size': [(6,3)],#,(2,2),(4,2),(3,3),(6, 3)],
    'pool_size': [(2,2)],#,(3, 2)],#,(3, 3),(4,3)],
    'num_layers': [3],  # Specify the range or list of possible values for the number of conv/pooling layers
    'num_dense_layers': [1]
}

param_grid = {
    'optimizer': ['adam'],#'adagrad','adadelta','sgd','adam',],
    'filters': [20,32],
    'kernel_size': [(6,3)],#,(2,2),(4,2),(3,3),(6, 3)],
    'num_layers': [3],  # Specify the range or list of possible values for the number of conv/pooling layers
}


In [274]:
##CNN model in a function
def create_CNN_model(dropout_rate=0,conv_activation='relu',kernel_size=(3,3), optimizer='adam',filters=32, num_layers=1,num_dense_layers=1):
    
    model = models.Sequential()
    num_layers = 3
    kernel_size = (6,3)
    for _ in range(num_layers):
        model.add(Conv2D(32*(_+1), kernel_size=kernel_size, activation='relu', input_shape=(height, width, channels)))
        model.add(MaxPooling2D(pool_size=(2,2), strides=2))
        
    model.add(Flatten())
    
    for _ in range(num_dense_layers):
        if dropout_rate != 0:
            model.add(Dropout(rate=dropout_rate))  
        model.add(Dense(units=64, activation=conv_activation))
    
    model.add(Dense(units=num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [275]:
# define function to display the results of the grid search
def display_cv_results(search_results):
    print('Best score = {:.4f} using {}'.format(search_results.best_score_, search_results.best_params_))
    means = search_results.cv_results_['mean_test_score']
    stds = search_results.cv_results_['std_test_score']
    params = search_results.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print('mean test accuracy +/- std = {:.4f} +/- {:.4f} with: {}'.format(mean, stdev, param))    
    


In [278]:
param_grid = {
    'batch_size': [32,64],
    'epochs': [50],
    'conv_activation': ['relu','tanh','sigmoid'],
    'dropout_rate': [0.10, 0.20, 0.30],
    'num_layers': [2,3],
    'filters': [20,32],
    'optimizer': ['sdg','rmsprop','adam'],
    'kernel_size':[(6,3),(4,2)],
    'num_dense_layers':[1,2]
}



X_tt, X_val, y_tt, y_val = train_test_split(X_train, y_train,
                                            test_size=.1,
                                            shuffle=True,
                                            random_state=41,
                                            stratify=y_train)

In [ ]:
from time import time
best_accuracy = 0.0
best_params = {}

# Perform GridSearchCV
start = time()
keras_classifier = KerasClassifier(model=create_CNN_model, verbose=1,dropout_rate=0.0,conv_activation='relu',kernel_size=(3,3), optimizer='adam',filters=32, num_layers=1,num_dense_layers=1)
                            #print(keras_classifier.model)
                            # Perform cross-validation
                            
# Use StratifiedKFold for cross-validation (assuming classification task)
#cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=2)

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=keras_classifier, param_grid=param_grid, cv=3,verbose=3)
grid_result = grid_search.fit(X_tt, to_categorical(y_tt))

                                                       
print('time for grid search = {:.0f} sec'.format(time()-start))
display_cv_results(grid_result)


Fitting 3 folds for each of 864 candidates, totalling 2592 fits
Epoch 1/50
7/7 [==============================] - 1s 83ms/step - loss: 7.2329 - accuracy: 0.3429
Epoch 2/50
7/7 [==============================] - 1s 80ms/step - loss: 0.9615 - accuracy: 0.5048
Epoch 3/50
7/7 [==============================] - 1s 80ms/step - loss: 0.9139 - accuracy: 0.5619
Epoch 4/50
7/7 [==============================] - 1s 81ms/step - loss: 0.8410 - accuracy: 0.6000
Epoch 5/50
7/7 [==============================] - 1s 81ms/step - loss: 0.7019 - accuracy: 0.6905
Epoch 6/50
7/7 [==============================] - 1s 83ms/step - loss: 0.6342 - accuracy: 0.7000
Epoch 7/50
7/7 [==============================] - 1s 81ms/step - loss: 0.6196 - accuracy: 0.6952
Epoch 8/50
7/7 [==============================] - 1s 79ms/step - loss: 0.5397 - accuracy: 0.7524
Epoch 9/50
7/7 [==============================] - 1s 79ms/step - loss: 0.4497 - accuracy: 0.8286
Epoch 10/50
7/7 [==============================] - 1s 79ms/step

7/7 [==============================] - 1s 80ms/step - loss: 0.0336 - accuracy: 0.9905
Epoch 32/50
7/7 [==============================] - 1s 81ms/step - loss: 0.0170 - accuracy: 0.9953
Epoch 33/50
7/7 [==============================] - 1s 81ms/step - loss: 0.0109 - accuracy: 1.0000
Epoch 34/50
7/7 [==============================] - 1s 83ms/step - loss: 0.0276 - accuracy: 0.9858
Epoch 35/50
7/7 [==============================] - 1s 82ms/step - loss: 0.0367 - accuracy: 0.9953
Epoch 36/50
7/7 [==============================] - 1s 83ms/step - loss: 0.0620 - accuracy: 0.9953
Epoch 37/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0161 - accuracy: 0.9953
Epoch 38/50
7/7 [==============================] - 1s 84ms/step - loss: 0.0462 - accuracy: 0.9953
Epoch 39/50
7/7 [==============================] - 1s 81ms/step - loss: 0.0235 - accuracy: 0.9953
Epoch 40/50
7/7 [==============================] - 1s 85ms/step - loss: 0.0154 - accuracy: 0.9953
Epoch 41/50
7/7 [===============

7/7 [==============================] - 1s 79ms/step - loss: 0.5965 - accuracy: 0.7095
Epoch 10/50
7/7 [==============================] - 1s 79ms/step - loss: 0.5575 - accuracy: 0.7143
Epoch 11/50
7/7 [==============================] - 1s 81ms/step - loss: 0.5459 - accuracy: 0.7476
Epoch 12/50
7/7 [==============================] - 1s 79ms/step - loss: 0.4470 - accuracy: 0.8000
Epoch 13/50
7/7 [==============================] - 1s 79ms/step - loss: 0.4135 - accuracy: 0.8429
Epoch 14/50
7/7 [==============================] - 1s 80ms/step - loss: 0.3650 - accuracy: 0.8571
Epoch 15/50
7/7 [==============================] - 1s 78ms/step - loss: 0.3101 - accuracy: 0.8905
Epoch 16/50
7/7 [==============================] - 1s 79ms/step - loss: 0.2776 - accuracy: 0.9048
Epoch 17/50
7/7 [==============================] - 1s 81ms/step - loss: 0.2724 - accuracy: 0.8714
Epoch 18/50
7/7 [==============================] - 1s 80ms/step - loss: 0.2336 - accuracy: 0.9286
Epoch 19/50
7/7 [===============

7/7 [==============================] - 1s 80ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 41/50
7/7 [==============================] - 1s 90ms/step - loss: 0.0113 - accuracy: 1.0000
Epoch 42/50
7/7 [==============================] - 1s 81ms/step - loss: 0.0052 - accuracy: 1.0000
Epoch 43/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0082 - accuracy: 0.9953
Epoch 44/50
7/7 [==============================] - 1s 79ms/step - loss: 0.0107 - accuracy: 1.0000
Epoch 45/50
7/7 [==============================] - 1s 79ms/step - loss: 0.0218 - accuracy: 0.9905
Epoch 46/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0173 - accuracy: 0.9953
Epoch 47/50
7/7 [==============================] - 1s 79ms/step - loss: 0.0089 - accuracy: 1.0000
Epoch 48/50
7/7 [==============================] - 1s 79ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 49/50
7/7 [==============================] - 1s 79ms/step - loss: 0.0131 - accuracy: 0.9953
Epoch 50/50
4/4 [===============

7/7 [==============================] - 1s 79ms/step - loss: 0.0683 - accuracy: 0.9810
Epoch 19/50
7/7 [==============================] - 1s 80ms/step - loss: 0.1058 - accuracy: 0.9667
Epoch 20/50
7/7 [==============================] - 1s 79ms/step - loss: 0.2077 - accuracy: 0.9286
Epoch 21/50
7/7 [==============================] - 1s 80ms/step - loss: 0.2291 - accuracy: 0.9429
Epoch 22/50
7/7 [==============================] - 1s 79ms/step - loss: 0.0961 - accuracy: 0.9619
Epoch 23/50
7/7 [==============================] - 1s 82ms/step - loss: 0.0932 - accuracy: 0.9810
Epoch 24/50
7/7 [==============================] - 1s 88ms/step - loss: 0.0671 - accuracy: 0.9762
Epoch 25/50
7/7 [==============================] - 1s 79ms/step - loss: 0.0349 - accuracy: 0.9905
Epoch 26/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0251 - accuracy: 1.0000
Epoch 27/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0086 - accuracy: 1.0000
Epoch 28/50
7/7 [===============

...

7/7 [==============================] - 1s 79ms/step - loss: 0.3586 - accuracy: 0.8389
Epoch 17/50
7/7 [==============================] - 1s 81ms/step - loss: 0.3455 - accuracy: 0.8626
Epoch 18/50
7/7 [==============================] - 1s 80ms/step - loss: 0.3311 - accuracy: 0.8768
Epoch 19/50
7/7 [==============================] - 1s 80ms/step - loss: 0.2676 - accuracy: 0.9005
Epoch 20/50
7/7 [==============================] - 1s 83ms/step - loss: 0.2572 - accuracy: 0.9194
Epoch 21/50
7/7 [==============================] - 1s 81ms/step - loss: 0.2204 - accuracy: 0.9147
Epoch 22/50
7/7 [==============================] - 1s 84ms/step - loss: 0.1881 - accuracy: 0.9289
Epoch 23/50
7/7 [==============================] - 1s 80ms/step - loss: 0.1741 - accuracy: 0.9194
Epoch 24/50
7/7 [==============================] - 1s 80ms/step - loss: 0.1255 - accuracy: 0.9621
Epoch 25/50
7/7 [==============================] - 1s 81ms/step - loss: 0.1036 - accuracy: 0.9668
Epoch 26/50
7/7 [===============

7/7 [==============================] - 1s 81ms/step - loss: 0.0663 - accuracy: 0.9810
Epoch 48/50
7/7 [==============================] - 1s 81ms/step - loss: 0.0325 - accuracy: 0.9905
Epoch 49/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0140 - accuracy: 1.0000
Epoch 50/50
4/4 [==============================] - 0s 21ms/step
[CV 3/3] END batch_size=32, conv_activation=relu, dropout_rate=0.1, epochs=50, filters=32, kernel_size=(4, 2), num_dense_layers=1, num_layers=2, optimizer=sdg;, score=0.790 total time=  29.2s
Epoch 1/50
7/7 [==============================] - 1s 79ms/step - loss: 12.5454 - accuracy: 0.3857
Epoch 2/50
7/7 [==============================] - 1s 80ms/step - loss: 1.0638 - accuracy: 0.4476
Epoch 3/50
7/7 [==============================] - 1s 79ms/step - loss: 0.9872 - accuracy: 0.5048
Epoch 4/50
7/7 [==============================] - 1s 81ms/step - loss: 0.8844 - accuracy: 0.5714
Epoch 5/50
7/7 [==============================] - 1s 80ms/step - loss: 0.

7/7 [==============================] - 1s 79ms/step - loss: 0.1649 - accuracy: 0.9431
Epoch 26/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0716 - accuracy: 0.9716
Epoch 27/50
7/7 [==============================] - 1s 79ms/step - loss: 0.0767 - accuracy: 0.9810
Epoch 28/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0623 - accuracy: 0.9858
Epoch 29/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0339 - accuracy: 0.9953
Epoch 30/50
7/7 [==============================] - 1s 81ms/step - loss: 0.0231 - accuracy: 0.9953
Epoch 31/50
7/7 [==============================] - 1s 78ms/step - loss: 0.0226 - accuracy: 0.9953
Epoch 32/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0158 - accuracy: 1.0000
Epoch 33/50
7/7 [==============================] - 1s 83ms/step - loss: 0.0136 - accuracy: 1.0000
Epoch 34/50
7/7 [==============================] - 1s 79ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 35/50
7/7 [===============

7/7 [==============================] - 1s 92ms/step - loss: 0.9317 - accuracy: 0.5571
Epoch 4/50
7/7 [==============================] - 1s 81ms/step - loss: 0.7919 - accuracy: 0.6333
Epoch 5/50
7/7 [==============================] - 1s 84ms/step - loss: 0.7007 - accuracy: 0.6857
Epoch 6/50
7/7 [==============================] - 1s 83ms/step - loss: 0.6215 - accuracy: 0.7476
Epoch 7/50
7/7 [==============================] - 1s 82ms/step - loss: 0.5949 - accuracy: 0.7190
Epoch 8/50
7/7 [==============================] - 1s 81ms/step - loss: 0.5918 - accuracy: 0.7619
Epoch 9/50
7/7 [==============================] - 1s 82ms/step - loss: 0.5334 - accuracy: 0.7667
Epoch 10/50
7/7 [==============================] - 1s 81ms/step - loss: 0.5182 - accuracy: 0.8000
Epoch 11/50
7/7 [==============================] - 1s 84ms/step - loss: 0.4509 - accuracy: 0.8143
Epoch 12/50
7/7 [==============================] - 1s 81ms/step - loss: 0.4372 - accuracy: 0.8190
Epoch 13/50
7/7 [=====================

7/7 [==============================] - 1s 89ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 35/50
7/7 [==============================] - 1s 81ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 36/50
7/7 [==============================] - 1s 79ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 37/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 38/50
7/7 [==============================] - 1s 81ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 39/50
7/7 [==============================] - 1s 80ms/step - loss: 7.8602e-04 - accuracy: 1.0000
Epoch 40/50
7/7 [==============================] - 1s 78ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 41/50
7/7 [==============================] - 1s 79ms/step - loss: 9.1924e-04 - accuracy: 1.0000
Epoch 42/50
7/7 [==============================] - 1s 80ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 43/50
7/7 [==============================] - 1s 82ms/step - loss: 0.0330 - accuracy: 0.9905
Epoch 44/50
7/7 [=======

In [266]:
grid_result.best_score_

0.740506329113924

<span style="font-family: 'Times New Roman'; font-size: 18px;">


    
</span>

